In [10]:
# !pip install flashrank
# !pip install chromadb
# !pip install openai langchain
# !pip install tiktoken

In [11]:
def check_ver(list):
    for i in range(len(list)):
        out = f"{list[i]} version : {list[i].__version__}"
        print(out)

In [12]:
import pandas as pd

import flashrank
from flashrank import Ranker, RerankRequest

import chromadb
from chromadb.utils import embedding_functions

import langchain
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

check_ver([pd,chromadb,langchain])

<module 'pandas' from '/Users/hikarii/anaconda3/lib/python3.10/site-packages/pandas/__init__.py'> version : 2.0.3
<module 'chromadb' from '/Users/hikarii/anaconda3/lib/python3.10/site-packages/chromadb/__init__.py'> version : 0.4.21
<module 'langchain' from '/Users/hikarii/anaconda3/lib/python3.10/site-packages/langchain/__init__.py'> version : 0.0.333


In [13]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv()) # read local .env file

openai_api_key = os.getenv('OPENAI_API_KEY_H')

# FlashRank

In [14]:
ranker = Ranker(model_name="ms-marco-MultiBERT-L-12")

In [15]:
# Metadata is optional, Id can be your DB ids from your retrieval stage or simple numeric indices.
query = "How to speedup LLMs?"
passages = [
   {
      "id":1,
      "text":"Introduce *lookahead decoding*: - a parallel decoding algo to accelerate LLM inference - w/o the need for a draft model or a data store - linearly decreases # decoding steps relative to log(FLOPs) used per decoding step.",
      "meta": {"additional": "info1"}
   },
   {
      "id":2,
      "text":"LLM inference efficiency will be one of the most crucial topics for both industry and academia, simply because the more efficient you are, the more $$$ you will save. vllm project is a must-read for this direction, and now they have just released the paper",
      "meta": {"additional": "info2"}
   },
   {
      "id":3,
      "text":"There are many ways to increase LLM inference throughput (tokens/second) and decrease memory footprint, sometimes at the same time. Here are a few methods I’ve found effective when working with Llama 2. These methods are all well-integrated with Hugging Face. This list is far from exhaustive; some of these techniques can be used in combination with each other and there are plenty of others to try. - Bettertransformer (Optimum Library): Simply call `model.to_bettertransformer()` on your Hugging Face model for a modest improvement in tokens per second. - Fp4 Mixed-Precision (Bitsandbytes): Requires minimal configuration and dramatically reduces the model's memory footprint. - AutoGPTQ: Time-consuming but leads to a much smaller model and faster inference. The quantization is a one-time cost that pays off in the long run.",
      "meta": {"additional": "info3"}

   },
   {
      "id":4,
      "text":"Ever want to make your LLM inference go brrrrr but got stuck at implementing speculative decoding and finding the suitable draft model? No more pain! Thrilled to unveil Medusa, a simple framework that removes the annoying draft model while getting 2x speedup.",
      "meta": {"additional": "info4"}
   },
   {
      "id":5,
      "text":"vLLM is a fast and easy-to-use library for LLM inference and serving. vLLM is fast with: State-of-the-art serving throughput Efficient management of attention key and value memory with PagedAttention Continuous batching of incoming requests Optimized CUDA kernels",
      "meta": {"additional": "info5"}
   }
]

rerankrequest = RerankRequest(query=query, passages=passages)
results = ranker.rerank(rerankrequest)
for i in results : print(i)

{'id': 4, 'text': 'Ever want to make your LLM inference go brrrrr but got stuck at implementing speculative decoding and finding the suitable draft model? No more pain! Thrilled to unveil Medusa, a simple framework that removes the annoying draft model while getting 2x speedup.', 'meta': {'additional': 'info4'}, 'score': 0.9572788}
{'id': 3, 'text': "There are many ways to increase LLM inference throughput (tokens/second) and decrease memory footprint, sometimes at the same time. Here are a few methods I’ve found effective when working with Llama 2. These methods are all well-integrated with Hugging Face. This list is far from exhaustive; some of these techniques can be used in combination with each other and there are plenty of others to try. - Bettertransformer (Optimum Library): Simply call `model.to_bettertransformer()` on your Hugging Face model for a modest improvement in tokens per second. - Fp4 Mixed-Precision (Bitsandbytes): Requires minimal configuration and dramatically redu

## ChromaDB

In [42]:
## clear existed database

!rm -rf chromadb

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [17]:
client = chromadb.PersistentClient(path="chromaDB/")

In [18]:
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key=openai_api_key,
                model_name="text-embedding-ada-002"
            )

In [19]:
collection = client.get_or_create_collection(name="mitsu", embedding_function=openai_ef)

### Text loading

In [20]:
df = pd.read_csv("rerank_dataset.csv", index_col=0)

df.tail(3)

,questions,answer,doc_page,doc_content,doc_source,label
90,ต้องทำอย่างไรเมื่อความเร็วรถลดลงต่ำกว่าประมาณ ...,"เมื่อความเร็วรถลดลงต่ำกว่าประมาณ 30 กม./ชม., ค...",127,แผงหน้าปัดและอุปกรณ์ควบคุม 5 \nคันสวิตช์ไฟเล...,docs/Text_PDF_.pdf,"['ลดลงตํ่ากว่า', 'E00536800021', 'งาน', 'โปรด'..."
91,ระบบ FCM อาจไม่ทำงานในสภาวะใดบ้าง?,ระบบ FCM อาจไม่ทำงานในสภาวะดังต่อไปนี้:\n- รถค...,213,○ ระบบ FCM อาจไม่ทํางานหากรถคันหน้ามีลักษณะแคบ...,docs/Text_PDF_.pdf,"['งาน', 'อาจทํา', 'ระหว่างรถ', 'อาจตรวจ', 'จับ..."
92,ตัวแสดงโหมดการขับเคลื่อน 4 ล้อแสดงสถานะการเปลี...,ตัวแสดงโหมดการขับเคลื่อน 4 ล้อจะสว่างตามตำแหน่...,180,ตัวแสดงโหมดการขับเคลื่อน 4 ล้อและตัวแสดงการทํ...,docs/Text_PDF_.pdf,"['E00606001774', 'งาน', '2WD4WD', 'จะสว่าง', '..."


In [21]:
# prepare chunks to be added to collection

doc = [df.doc_content[i] for i in range(len(df))]
meta = [{"label":df.label[i]} for i in range(len(df))]
idd = ["id{a}".format(a=i+1) for i in range(len(df))]

doc_embed = openai_ef(doc)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [22]:
collection.add(
    embeddings = doc_embed,
    documents = doc,
    metadatas = meta,
    ids = idd
)

In [23]:
client.list_collections() # list tables

[Collection(name=mitsu)]

In [24]:
collection.count()

93

In [25]:
collection.get(ids=["id1"],
    where=None)

{'ids': ['id1'],
 'embeddings': None,
 'metadatas': [{'label': "['ดับเบิลแค็บ', 'งาน', 'ขับ', 'ขับการ', 'ซํ้า']"}],
 'documents': ['และดึงสวิตช์ขึ้นเพื่อปิดกุญแจอัจฉริยะ หมายเหตุ -  การเปิดปิดซํ้าๆ ขณะที่เครื่องยนต์ไม่ทํางานจะเป็นการสิ้นเปลืองแบตเตอรี่พยายามอย่าใช้สวิตช์กระจกยกเว้นในขณะที่เครื่องยนต์กําลังทํางาน -  กระจกหน้าต่างประตูหลังจะเปิดเพียงครึ่งเดียวเท่านั้น (ดับเบิลแค็บ) สวิตช์ล็อก เมื่อกดสวิตช์ล็อก (1) ไฟแสดง (2) จะสว่างขึ้น และสวิตช์ด้านผู้โดยสารจะไม่สามารถใช้เปิดหรือปิดกระจกหน้าต่างประตูได้ และสวิตช์ด้านคนขับจะไม่สามารถเปิดหรือปิดกระจกหน้าต่างประตูอื่นได้นอกจากกระจกหน้าต่างประตูด้านคนขับการปลดล็อก ให้กดสวิตช์ล็อกอีกครั้ง และไฟแสดงจะดับลงกุญแจอัจฉริยะ'],
 'uris': None,
 'data': None}

### Query

In [26]:
## simple query

results = collection.query(
    query_texts=["สวิตช์ล็อกทำงานอย่างไร?"],
    n_results=2
)

results

{'ids': [['id1', 'id77']],
 'distances': [[0.3037891551003212, 0.32300425397003935]],
 'metadatas': [[{'label': "['ดับเบิลแค็บ', 'งาน', 'ขับ', 'ขับการ', 'ซํ้า']"},
   {'label': "['งาน', 'ขับ', 'ซํ้า', 'ขับได้', 'สวิตช์']"}]],
 'embeddings': None,
 'documents': [['และดึงสวิตช์ขึ้นเพื่อปิดกุญแจอัจฉริยะ หมายเหตุ -  การเปิดปิดซํ้าๆ ขณะที่เครื่องยนต์ไม่ทํางานจะเป็นการสิ้นเปลืองแบตเตอรี่พยายามอย่าใช้สวิตช์กระจกยกเว้นในขณะที่เครื่องยนต์กําลังทํางาน -  กระจกหน้าต่างประตูหลังจะเปิดเพียงครึ่งเดียวเท่านั้น (ดับเบิลแค็บ) สวิตช์ล็อก เมื่อกดสวิตช์ล็อก (1) ไฟแสดง (2) จะสว่างขึ้น และสวิตช์ด้านผู้โดยสารจะไม่สามารถใช้เปิดหรือปิดกระจกหน้าต่างประตูได้ และสวิตช์ด้านคนขับจะไม่สามารถเปิดหรือปิดกระจกหน้าต่างประตูอื่นได้นอกจากกระจกหน้าต่างประตูด้านคนขับการปลดล็อก ให้กดสวิตช์ล็อกอีกครั้ง และไฟแสดงจะดับลงกุญแจอัจฉริยะ',
   'ยู่ ให้ดึงสวิตช์กระจกหน้าต่างประตูด้านคนขับขึ้นซํ้าๆ  จนกระทั่งกระจกหน้าต่างปิดสนิท  หลังจากนั้นให้ปล่อยสวิตช์  \nดึงสวิตช์ค้างไว้อีกครั้งหนึ่งอย่างน้อย  1 วินาทีแล้วปล่อยสวิตช์  คุณก็จะสามาร

In [27]:
### Passing collection2 to langchain

embeddings = OpenAIEmbeddings( api_key=openai_api_key )
# vectordb = Chroma.from_documents(texts, embeddings)

vectordb = Chroma(
    client=client,
    collection_name="mitsu",
    embedding_function=embeddings,
)

retriever = vectordb.as_retriever(search_type="similarity", search_kwargs={"k": 15})

In [37]:
# query = "การเปิดปิดสวิตช์ซํ้าๆ ขณะที่เครื่องยนต์ไม่ทํางานจะเกิดอะไรขึ้น"
query = "รถยางแตกควรทำยังไง"

retrieved_docs = retriever.invoke(
    query
)

print(f"number of documents : {len(retrieved_docs)}\
      \nfirst document : {retrieved_docs[0].page_content}\
      \nmetadata : {retrieved_docs[0].metadata}")

number of documents : 15      
first document : เส้นคาดการณ์ แนวทางรถอาจแสดงไม่ถูกต้อง     • บนถนนที่ปกคลุมด้วยหิมะหรือลื่นอาจมีความแตกต่าง ระหว่างเส้นคาดการณ์แนวทางรถกับเส้นแนวทางรถจริง      
metadata : {'label': "['', 'ถนน', 'ปกคลุม', 'การณ์แนวทาง', 'แสดงไม่']"}


### convert datatype for FlashRank

In [38]:
def convert_retrieved_doc(retrieved_docs):
  passages = []
  for i,con in enumerate(retrieved_docs):
    for_rerank = {
                      "id":i,
                      "text":con.page_content,
                      "meta": con.metadata
                  }
    passages.append(for_rerank)
  return passages

In [39]:
passages = convert_retrieved_doc(retrieved_docs)

print(query)

รถยางแตกควรทำยังไง


Re-ranking results

In [40]:
rerankrequest = RerankRequest(query=query, passages=passages)
results = ranker.rerank(rerankrequest)
for i in range(4) : print(results[i])

{'id': 7, 'text': '○ขับขี่บนพื้นถนนที่เป็นกรวดหรือปกคลุมด้วยหิมะ ○ขับขี่โดยใช้โซ่พันล้อ ○ขับขี่บนพื้นถนนที่เป็นหลุมเป็นบ่อหรือสูงตํ่าแตกต่างกัน ○ขับขี่บนพื้นผิวถนนที่ไม่เรียบ ●ระบบ ABS ไม่ได้จํากัดอยู่เฉพาะในสถานการณ์ที่มีการเหยียบเบรกอย่างกะทันหันเท่านั้น ระบบนี้ยังอาจช่วยป้องกันไม่ให้ล้อรถล็อกขณะที่คุณขับผ่านฝาท่อ แผ่นเหล็กบนหน้าถนน รอยบนถนน หรือพื้นผิวถนนที่ไม่เรียบอื่นๆ ●เมื่อมีการใช้งานระบบ ABS คุณอาจรู้สึกได้ว่าแป้นเบรกสะเทือนเป็นจังหวะและตัวรถและพวงมาลัยสั่นสะเทือน และยังอาจรู้สึกได้ว่าแป้นเบรกเกิดแรงต้านขณะเหยียบเบรก ในสถานการณ์เช่นนี้ ให้เหยียบแป้นเบรกอย่างหนักแน่น อย่ากดเบรกแล้วปล่อยซํ้าๆ', 'meta': {'label': "['งาน', 'กรวด', 'ท่อ', 'ขับ', 'เมื่อ']"}, 'score': 0.56392366}
{'id': 0, 'text': 'เส้นคาดการณ์ แนวทางรถอาจแสดงไม่ถูกต้อง     • บนถนนที่ปกคลุมด้วยหิมะหรือลื่นอาจมีความแตกต่าง ระหว่างเส้นคาดการณ์แนวทางรถกับเส้นแนวทางรถจริง', 'meta': {'label': "['', 'ถนน', 'ปกคลุม', 'การณ์แนวทาง', 'แสดงไม่']"}, 'score': 0.43370095}
{'id': 4, 'text': 'ให้ขับรถช้าๆ ตามปกติหลังจากตรวจดูแล้วว่า

In [41]:
retrieved_docs[:4]

[Document(page_content='เส้นคาดการณ์ แนวทางรถอาจแสดงไม่ถูกต้อง     • บนถนนที่ปกคลุมด้วยหิมะหรือลื่นอาจมีความแตกต่าง ระหว่างเส้นคาดการณ์แนวทางรถกับเส้นแนวทางรถจริง', metadata={'label': "['', 'ถนน', 'ปกคลุม', 'การณ์แนวทาง', 'แสดงไม่']"}),
 Document(page_content='ข้อควรระวัง ● หํามลากรถเกียร์อัตโนมัติโดยยกล้อหน้าขึ้น (และล้อหลังอยู่บนพื้น) (แบบ B) เมื่อนํ้ามันเกียร์อยู่', metadata={'label': "['ควรระวัง', 'ยกล้อหน้า', 'ข้อ', 'เกียร์', 'เกียร์อัตโนมัติ']"}),
 Document(page_content='การเร่งหรือลดความเร็วทันที) ■เมื่อรถและคนเดินถนนอยู่เยี้องจากเส้นทางไปข้างหน้าของรถ ■หากความเร็วระหว่างรถสองคันต่างกันเล็กน้อย ■ประมาณ 15 วินาทีหลังจากสตาร์ทเครื่องยนต์ ■หากรถคันหน้าหรือรถที่ใกล้เข้ามามีรูปทรงเฉพาะหรือผิดปกติมีระยะห่างระหว่างความสูงและตํ่ามาก หรือการบรรทุกสัมภาระผิดปกติหรือแคบ (ตัวอย่างเช่น รถจักรยานยนต์) คําเตือน ■เมื่อรถหรือคนเดินถนนอยู่ใกล้เครื่องหมายจราจร บริเวณที่เกิดการสะท้อน (ตัวอย่างเช่น นํ้าบนถนน) หรืออยู่ในเงา ■เมื่อคนเดินถนนหลายคนรวมตัวกัน ■เมื่อรถหรือวัตถุอื่นๆ ทําให้การมองคนเดินถนนไม

# Simple Neural Networks